   # Код для поиска вакансий

In [1]:
path = 'C:/Users/xazzz/Documents/'

In [2]:
# Библиотека для работы с HTTP-запросами. Будем использовать ее для обращения к API HH
import requests
 
# Пакет для удобной работы с данными в формате json
import json
 
# Модуль для работы со значением времени
import time
 
# Модуль для работы с операционной системой. Будем использовать для работы с файлами
import os

In [5]:
def getPage(page = 0):
    """
    Создаем метод для получения страницы со списком вакансий.
    Аргументы:
        page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
    """
     
    # Справочник для параметров GET-запроса
    params = {
        'text': 'Аналитик SQL Python NOT !системный NOT !разработчик NOT !QA NOT !ведущий NOT !руководитель',
        'area': 3, # Поиск ощуществляется по вакансиям города Екатеринбург
        'period': 3, # За какое количество дней
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }
     
     
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data

In [6]:
# Считываем первые 2000 вакансий
for page in range(0, 20):
     
    # Преобразуем текст ответа запроса в справочник Python
    jsObj = json.loads(getPage(page))
     
    # Сохраняем файлы в папку {путь до текущего документа со скриптом}\docs\pagination
    # Определяем количество файлов в папке для сохранения документа с ответом запроса
    # Полученное значение используем для формирования имени документа
    nextFileName = path + '/Untitled Folder/{}.json'.format(len(os.listdir(path + '/Untitled Folder/')))
     
    # Создаем новый документ, записываем в него ответ запроса, после закрываем
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False))
    f.close()
     
    # Проверка на последнюю страницу, если вакансий меньше 2000
    if (jsObj['pages'] - page) <= 1:
        break
     
    # Необязательная задержка, но чтобы не нагружать сервисы hh, оставим. 5 сек мы может подождать
    time.sleep(0.25)
     
print('Старницы поиска собраны')

Старницы поиска собраны


In [7]:
 # Получаем перечень ранее созданных файлов со списком вакансий и проходимся по нему в цикле 
a = [i for i in os.listdir(path + '/Untitled Folder/') if not i.startswith('.')]
new_list = []
for fl in a:
     
    # Открываем файл, читаем его содержимое, закрываем файл
    f = open(path + '/Untitled Folder/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
     
    # Преобразуем полученный текст в объект справочника (создаю список с url вакансий - new.list)
    jsonObj = json.loads(jsonText)
    jsonObj_1 = jsonObj['items']
    
    #i = 0
    #while i != len(jsonObj_1):
    for i in range(len(jsonObj_1)):
        for key, value in jsonObj_1[i].items():
            if key == 'alternate_url':
                new_list.append(value)
        i += 1
    print(len(jsonObj_1))

7


# Код для отправки сообщения мне на телеграм

In [23]:
import requests
import json
from urllib.parse import urlencode

In [9]:
token = '5255431708:AAHFx9iDUF5Xq5xjiDZlvbmmEMHIZ2H-vlo'

In [16]:
api_pattern = f'https://api.telegram.org/bot{token}'

In [17]:
update = f'{api_pattern}/getUpdates'

In [18]:
info = requests.get(update)

In [19]:
info.status_code

200

In [20]:
info.json()

{'ok': True,
 'result': [{'update_id': 897993610,
   'message': {'message_id': 63,
    'from': {'id': 487621444,
     'is_bot': False,
     'first_name': 'Миляуша',
     'username': 'negolub',
     'language_code': 'ru'},
    'chat': {'id': 487621444,
     'first_name': 'Миляуша',
     'username': 'negolub',
     'type': 'private'},
    'date': 1645031823,
    'text': '/start',
    'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}]}

In [21]:
chat_id = 487621444

In [24]:
for i in new_list:
    message = i
    params = {'chat_id': chat_id, 'text': message}
    base_url = f'https://api.telegram.org/bot{token}/'
    url = base_url + 'sendMessage?' + urlencode(params)
    resp = requests.get(url)

NameError: name 'base_url' is not defined